In [ ]:
import time
import json
import csv
from abc import ABC, abstractmethod
from functools import wraps
from typing import List, Iterator, Generator

In [ ]:
# Descriptor 
# Validating Marks between 0 and 100 
class ValidateMarks:
    def __get__(self, instance, owner):
        return instance._marks

    def __set__(self, instance, value):
        for v in value:
            if v < 0 or v > 100:
                raise ValueError("Error: Marks should be between 0 and 100")
        instance._marks = value

# Checking Salary is non-negative and immutable after initial set      
class ValidateSalary:
    def __get__(self, instance, owner):
        return instance._salary

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Salary must be non-negative")
        
        if hasattr(instance, "_salary"):
            raise AttributeError("Salary cannot be changed directly")
        instance._salary = value

In [ ]:
# Decorators
# Decorator for access control
def admin_only(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        if not hasattr(wrapper, 'is_admin'):
            wrapper.is_admin = True          
        if not wrapper.is_admin:
            print("Access Denied: Admin privileges required")
            return None
        return func(*args, **kwargs)
    return wrapper

# Decorator for log method execution
def log_execution(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f"[LOG] Method {func.__name__}() executed successfully")
        return result
    return wrapper

# Decorator for measuring execution time
def time_performance(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print("Execution Time",func.__name__,"took",end-start,"seconds")
        return result
    return wrapper

In [ ]:
# Iterator
# Iterator for traversing students
class StudentIterator:
    def __init__(self, students):
        self.students = students
        self.index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index < len(self.students):
            student = self.students[self.index]
            self.index += 1
            return student
        raise StopIteration

In [ ]:
# Generator to yield student records batch-wise
def student_record_generator(students, batch_size: int = 2) -> Generator:
    for i in range(0, len(students), batch_size):
        yield students[i:i + batch_size]

In [ ]:
# Abstract Class
class Person(ABC):
    def __init__(self, pid, name, dept):
        self.pid = pid
        self.name = name
        self.dept = dept
    
    @abstractmethod
    def get_details(self):
        pass
    
    @abstractmethod
    def calculate_performance(self):
        pass
    
    def __del__(self):
        print(f"{self.__class__.__name__} object {self.name} is being deleted")

In [ ]:
# Inherits Abstract Class 
class Student(Person):
    marks = ValidateMarks()
    def __init__(self, sid, name, dept, semester, marks):
        super().__init__(sid, name, dept)
        self.semester = semester
        self.marks = marks
        self.enrolled_courses = []
    
    def get_details(self):
        print("Student Details:")
        print("----------------")
        print("Name:       " + self.name)
        print("Role:       Student")
        print("Department: " + self.dept)
    
    def calculate_performance(self):
        if not self.marks:
            print("Invalid Marks")
            return
        
        avg = sum(self.marks) / len(self.marks)
         
        if avg > 90:
            grade = 'A'
        elif avg > 80:
            grade = 'B' 
        elif avg > 70:
            grade = "B"
        elif avg > 60:
            grade = "C"
        elif avg > 50:
            grade = "D"
        else:
            grade = "F"
        return round(avg,2),grade
    
    # Enrolling to a course
    def enroll_course(self, course):
        if course not in self.enrolled_courses:
            self.enrolled_courses.append(course)
            return True
        return False
    
    def __gt__(self, other):
        if not isinstance(other, Student):
            return NotImplemented
        my_avg, _ = self.calculate_performance()
        other_avg, _ = other.calculate_performance()
        return my_avg > other_avg

In [ ]:
class Faculty(Person):
    salary = ValidateSalary()
    def __init__(self, fid, name, dept, salary):
        super().__init__(fid, name, dept)
        self.salary = salary
        self.assigned_courses = []
    
    def get_details(self):
        print("Faculty Details:")
        print("----------------")
        print("Name:       " + self.name)
        print("Role:       Faculty")
        print("Department: " + self.dept)
    
    # Assigning course to faculty
    def assign_course(self, course):
        if course not in self.assigned_courses:
            self.assigned_courses.append(course)
            return True
        return False
    
    def calculate_performance(self):
        return len(self.assigned_courses)
    
    @admin_only
    def update_salary(self, new_salary):
        if new_salary < 0:
            print("Salary cannot be negative")
        else:
            self._salary = new_salary 
    
    @admin_only
    def get_salary(self):
        return self.salary

In [ ]:
class Course:
    def __init__(self, ccd, name, credits, faculty):
        self.ccd = ccd
        self.name = name
        self.credits = credits
        self.faculty = faculty
        self.enrolled_students = []
    
    # Enrolling a student to a course
    def enroll_student(self, student):
        if student not in self.enrolled_students:
            self.enrolled_students.append(student)
            student.enroll_course(self)
            return True
        return False
    

In [ ]:
class Department:
    def __init__(self, dept):
        self.dept = dept
        self.students = []
        self.faculties = []
        self.courses = []
    
    def add_student(self, student):
        self.students.append(student)
    
    def add_faculty(self, faculty):
        self.faculties.append(faculty)
    
    def add_course(self, course):
        self.courses.append(course)

In [ ]:
class SmartUniversityManagementSystem:
    # Using Dictionaries to store data
    def __init__(self):
        self.students = {}
        self.faculties = {}
        self.courses = {}
        self.departments = {}
        
    def add_student(self, sid, name, dept, semester, marks):
        try:
            if sid in self.students:
                raise ValueError("Error: This Student already exists")
            
            s = Student(sid, name, dept, semester, marks)
            self.students[sid] = s
        
            print("\nStudent Created Successfully")
            print("------------------------------")
            print("ID        :",sid)
            print("Name      :",name)
            print("Department:",dept)
            print("Semester  :",semester)
        except ValueError as e:
            print(str(e))
    
    def add_faculty(self, fid, name, dept, salary):
        try:
            if fid in self.faculties:
                raise ValueError("Error: This Faculty already exists")
            
            f = Faculty(fid, name, dept, salary)
            self.faculties[fid] = f
            
            print("\nFaculty Created Successfully")
            print("------------------------------")
            print("ID        :",fid)
            print("Name      :",name)
            print("Department:",dept)
            
        except ValueError as e:
            print(str(e))
    
    def add_course(self, ccd, name, credits, fid):
        try:
            if ccd in self.courses:
                raise ValueError("Error: Course code already exists")
            
            f = self.faculties.get(fid)
            c = Course(ccd, name, credits, f)
            self.courses[ccd] = c
            
            print("\nCourse Added Successfully")
            print("---------------------------")
            print("Course Code : ",ccd)
            print("Course Name : ",name)
            print("Credits     : ",credits)
            print("Faculty     : ",f.name)
            
        except ValueError as e:
            print(str(e))
    
    def enroll_student_to_course(self, sid, ccd):
        try:
            s = self.students.get(sid)
            c = self.courses.get(ccd)
            
            if not s:
                raise ValueError("Error: Student not found")
            if not c:
                raise ValueError("Error: Course not found")
            
            if c.enroll_student(s):
                print("\nEnrollment Successful")
                print("-----------------------")
                print("Student Name : ",s.name)
                print(f"Course      : ",c.name)
            else:
                print(s.name,"has already enrolled in ",c.name)
                
        except ValueError as e:
            print(str(e))
    
    def calculate_student_performance(self, sid):
        try:
            s = self.students.get(sid)
            if not s:
                raise ValueError("Error: Student not found")
            
            average, grade = s.calculate_performance()
            
            print("\nStudent Performance Report")
            print("-" * 32)
            print("Student Name : ",s.name)
            print("Marks        : ",s.marks)
            print("Average      : ",average)
            print("Grade        : ",grade)
            
        except ValueError as e:
            print(str(e))
            
    def compare_students(self, sid1, sid2):
        """Compare two students using operator overloading"""
        try:
            student1 = self.students.get(sid1)
            student2 = self.students.get(sid2)
            
            if not student1 or not student2:
                raise ValueError("Error: One or both students not found")
            
            print("\nCompare Two Students")
            print("----------------------") 
            result = student1 > student2
            print(f"{student1.name} > {student2.name} : {result}")
            
        except ValueError as e:
            print(str(e))
            
    @time_performance
    def generate_reports(self):
        try:
            with open('students_report.csv', 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['ID', 'Name', 'Department', 'Average', 'Grade'])                
                for student in self.students.values():
                    average, grade = student.calculate_performance()
                    writer.writerow([student.pid, student.name, student.dept, average, grade])  # ← FIX HERE
        
            print("\nCSV Report Generated: students_report.csv")
        
            students_data = []
            for student in self.students.values():
                students_data.append({
                    "id": student.pid,        # ← FIX HERE
                    "name": student.name,
                    "department": student.dept,  # ← FIX HERE
                    "semester": student.semester,
                    "marks": student.marks
                })       
            with open('students.json', 'w') as jsonfile:
                json.dump(students_data, jsonfile, indent=2)
        
            print("Student data successfully saved to students.json")
        
        except IOError as e:
            print(f"Error: File access error - {str(e)}")
                 

In [ ]:
def main():
    smus = SmartUniversityManagementSystem() 
    while(True):
        print("\n    Smart University Management System ")
        print("-----------------------------------------") 
        print("1 -> Add Student")
        print("2 -> Add Faculty")
        print("3 -> Add Course")
        print("4 -> Enroll Student to Course")
        print("5 -> Calculate Student Performance")
        print("6 -> Compare Two Students")
        print("7 -> Generate Reports")
        print("8 -> Exit")
        
        try:
            scanner = int(input("Enter your choice: ")) 
            if scanner == 1:
                sid = input("Student ID: ")
                name = input("Student Name: ")
                dept = input("Department: ")
                semester = int(input("Semester: "))
                marks = list(map(int, input("Marks (5 subjects separated by space): ").split()))
                smus.add_student(sid, name, dept, semester, marks)
        
            elif scanner == 2:
                    fid = input("Faculty ID: ")
                    name = input("Faculty Name: ")
                    dept = input("Department: ")
                    salary = float(input("Monthly Salary: "))
                    smus.add_faculty(fid, name, dept, salary)
        
            elif scanner == 3:
                    ccd = input("Course Code: ")
                    name = input("Course Name: ")
                    credits = int(input("Credits: "))
                    fid = input("Faculty ID: ")
                    smus.add_course(ccd, name, credits, fid)
        
            elif scanner == 4:
                    sid = input("Student ID: ")
                    ccd = input("Course Code: ")
                    smus.enroll_student_to_course(sid, ccd)
        
            elif scanner == 5:
                sid = input("Student ID: ")
                smus.calculate_student_performance(sid)
        
            elif scanner == 6:
                sid1 = input("First Student ID: ")
                sid2 = input("Second Student ID: ")
                smus.compare_students(sid1, sid2)
        
            elif scanner == 7:
                smus.generate_reports()
        
            elif scanner == 8:
                print("Thank you for using Smart University Management System")
                break
            else:
                print("\nInvalid choice! Please try again.")
                
        except ValueError as e:
            print(f"\nError: Invalid input - {str(e)}")
        except Exception as e:
            print(f"\nError: {str(e)}")   

In [ ]:
if __name__ == "__main__":
    main()